<a href="https://colab.research.google.com/github/trista-paul/DS-Unit-4-Sprint-2-NLP/blob/master/Trista_LS_DS_424_Word_Embeddings_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Use Word2Vec to train your own model on a dataset.

1) **Optional** - Find your own dataset of documents to train you model on. You are going to need a lot of data, so it's probably not realistic to scrape data for this assignment given the time constraints that we're working under. Try to find a dataset that has > 5000 documents.

- If you can't find a dataset to use try this one: <https://www.kaggle.com/c/quora-question-pairs>

2) Clean/Tokenize the documents.

3) Vectorize the model using Word2Vec and explore the results using each of the following at least one time:

- your_model.wv.most_similar()
- your_model.wv.similarity()
- your_model.wv.doesn't_match()

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

!kaggle competitions download -c quora-question-pairs

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/
  0% 0.00/4.95M [00:00<?, ?B/s]
100% 4.95M/4.95M [00:00<00:00, 45.5MB/s]
 80% 17.0M/21.2M [00:00<00:00, 30.8MB/s]
100% 21.2M/21.2M [00:00<00:00, 60.9MB/s]
100% 112M/112M [00:01<00:00, 57.3MB/s] 
100% 112M/112M [00:01<00:00, 61.4MB/s]


In [3]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [5]:
!pip install -U gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.7.2)


In [6]:
import pandas as pd
import numpy as np
import string
import re

import gensim
from gensim.models.word2vec import Word2Vec

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [10]:
print(train.shape)
train.head()
#and I immediately regret using a quora dataset.
#Time to not look at the text I'm working with

(404290, 6)


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [12]:
#is_duplicate is our y
train['is_duplicate'].value_counts(normalize=True)

0    0.630802
1    0.369198
Name: is_duplicate, dtype: float64

In [0]:
#force everything to be str
train['question1'] = train['question1'].astype(str)
train['question2'] = train['question2'].astype(str)

In [0]:
#tokenizing, cleaning, and lemmatization from previous doc

from string import punctuation
table = str.maketrans('', '', punctuation)
stop_words = stopwords.words('english')

def process_text(text):
    #sep by word
    words = word_tokenize(text)

    #lowercase
    words = [word.lower() for word in words]

    #remove non alphanumeric characters
    words = [word.translate(table) for word in words]
    words = [word for word in words if word.isalpha()]

    #remove stopwords
    words = [word for word in words if not word in stop_words]
    
    #lemmatize (remove prefixes and suffixes)
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    #final reformat
    words = " ".join(words)
    
    return words

train['question1'] = train['question1'].apply(process_text)

In [0]:
#oh god this takes forever when made competently
#still better than copypasting ten cells in order to split into lists and back
train['question2'] = train['question2'].apply(process_text)

test['question1'] = test['question1'].astype(str)
test['question2'] = test['question2'].astype(str)
test['question1'] = test['question1'].apply(process_text)
test['question2'] = test['question2'].apply(process_text)

In [0]:
#apparently it kept lots of individual characters and foriegn characters
def rm_chara(text):
  words = word_tokenize(text)
  words = [word for word in words if len(word) > 1]
  return words

train['question1'] = train['question1'].apply(rm_chara)
train['question2'] = train['question2'].apply(rm_chara)

In [17]:
train.head()
#mentally lonely solve...if that isn't the platonic ideal quora

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,step step guide invest share market india,step step guide invest share market,0
1,1,3,4,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,0
2,2,5,6,increase speed internet connection using vpn,internet speed increased hacking dns,0
3,3,7,8,mentally lonely solve,find remainder math math divided,0
4,4,9,10,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0


In [0]:
#min_count 5 and limit 100 break the vocab
word = Word2Vec(train['question1'])

In [34]:
word.wv.vocab.keys()

dict_keys(['step', 'guide', 'invest', 'share', 'market', 'india', 'story', 'kohinoor', 'diamond', 'increase', 'speed', 'internet', 'connection', 'using', 'vpn', 'mentally', 'lonely', 'solve', 'one', 'dissolve', 'water', 'sugar', 'salt', 'methane', 'carbon', 'di', 'oxide', 'astrology', 'capricorn', 'sun', 'cap', 'moon', 'rising', 'say', 'buy', 'good', 'geologist', 'use', 'instead', 'motorola', 'company', 'hack', 'charter', 'method', 'find', 'separation', 'slit', 'read', 'youtube', 'comment', 'make', 'physic', 'easy', 'learn', 'first', 'sexual', 'experience', 'like', 'law', 'change', 'status', 'student', 'visa', 'green', 'card', 'compare', 'immigration', 'canada', 'would', 'trump', 'presidency', 'mean', 'current', 'international', 'master', 'manipulation', 'girl', 'want', 'friend', 'guy', 'reject', 'many', 'quora', 'user', 'posting', 'question', 'readily', 'answered', 'google', 'best', 'digital', 'marketing', 'institution', 'banglore', 'rocket', 'look', 'white', 'causing', 'someone', 'je

In [35]:
word.wv.most_similar('strip') #I'm scared of context

[('sweater', 0.827745258808136),
 ('gourmet', 0.8214501142501831),
 ('lounge', 0.8213142156600952),
 ('bridesmaid', 0.8195719718933105),
 ('kent', 0.8160438537597656),
 ('blazer', 0.8134450912475586),
 ('reflector', 0.8110089302062988),
 ('del', 0.8104393482208252),
 ('tent', 0.8091695308685303),
 ('turquoise', 0.8091028332710266)]

In [36]:
word.wv.most_similar('purpose')
#our purpose is to shelf!

[('shelf', 0.6127689480781555),
 ('function', 0.6034682989120483),
 ('extraterrestrial', 0.5922471880912781),
 ('principle', 0.5884621143341064),
 ('indeen', 0.5858185887336731),
 ('expectancy', 0.5738992094993591),
 ('terrestrial', 0.5715219974517822),
 ('ultimate', 0.5703617334365845),
 ('importance', 0.5683416128158569),
 ('definition', 0.5681662559509277)]

In [38]:
word.wv.similarity('purpose', 'life') #life has very little to do with purpose

0.12579505

In [45]:
word.wv.doesnt_match("purpose life ultimate shelf".split())
#life is the least related concept in that list

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:858: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'life'

### Stretch Goals:

1) Use Doc2Vec to train a model on your dataset, and then provide model with a new document and let it find similar documents.

2) Download the pre-trained word vectors from Google. Access the pre-trained vectors via the following link: https://code.google.com/archive/p/word2vec

Load the pre-trained word vectors and train the Word2vec model

Examine the first 100 keys or words of the vocabulary

Outputs the vector representation for a select set of words - the words can be of your choice

Examine the similarity between words - the words can be of your choice

For example:

model.similarity('house', 'bungalow')

model.similarity('house', 'umbrella')

# Testing Doc2Vec with imports

In [50]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
TaggedDocument #a TaggedDocument object
common_texts #a list of lists of strings. Not joined.

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [51]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
documents #transformation of each list of strings to TaggedDocument

[TaggedDocument(words=['human', 'interface', 'computer'], tags=[0]),
 TaggedDocument(words=['survey', 'user', 'computer', 'system', 'response', 'time'], tags=[1]),
 TaggedDocument(words=['eps', 'user', 'interface', 'system'], tags=[2]),
 TaggedDocument(words=['system', 'human', 'system', 'eps'], tags=[3]),
 TaggedDocument(words=['user', 'response', 'time'], tags=[4]),
 TaggedDocument(words=['trees'], tags=[5]),
 TaggedDocument(words=['graph', 'trees'], tags=[6]),
 TaggedDocument(words=['graph', 'minors', 'trees'], tags=[7]),
 TaggedDocument(words=['graph', 'minors', 'survey'], tags=[8])]

In [0]:
doc = Doc2Vec(documents, vector_size=5, window=2,
                         min_count=1, workers=4)
vector = doc.infer_vector(["system", "response"]) #now new documents can be compared

In [53]:
vector

array([-0.07459321,  0.01781676,  0.03452338, -0.05933785,  0.04146539],
      dtype=float32)

# Applying Doc2Vec to Quora

In [0]:
questions = [TaggedDocument(doc, [i]) for i, doc in enumerate(train['question1'])]

In [0]:
doc = Doc2Vec(questions, vector_size=5, window=2,
                         min_count=1, workers=4)

In [0]:
question_comparison = doc.infer_vector(train['question2'][0])

In [62]:
question_comparison

array([-0.05419756, -0.02403936, -0.08838025,  0.0613296 , -0.03113589],
      dtype=float32)